##### Copyright 2018 Авторы TF-Агентов.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Обучите глубокую сеть Q с TF-агентами

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/agents/tutorials/1_dqn_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> Посмотреть на TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> Запустить в Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Посмотреть источник на GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/agents/docs/tutorials/1_dqn_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"> Скачать блокнот</a></td>
</table>

## Введение


В этом примере показано, как обучить [агента DQN (Deep Q Networks)](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) в среде Cartpole с использованием библиотеки TF-Agents.

![Среда обитания](https://raw.githubusercontent.com/tensorflow/agents/master/docs/tutorials/images/cartpole.png)

Он проведет вас через все компоненты конвейера Reinforcement Learning (RL) для обучения, оценки и сбора данных.

Чтобы запустить этот код в прямом эфире, нажмите ссылку «Запустить в Google Colab» выше.


## Настроить

Если вы не установили следующие зависимости, запустите:

In [0]:
!sudo apt-get install -y xvfb ffmpeg
!pip install 'gym==0.10.11'
!pip install 'imageio==2.4.0'
!pip install PILLOW
!pip install 'pyglet==1.3.2'
!pip install pyvirtualdisplay
!pip install tf-agents

In [0]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import q_network
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

In [0]:
tf.compat.v1.enable_v2_behavior()

# Set up a virtual display for rendering OpenAI gym environments.
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

In [0]:
tf.version.VERSION

## гиперпараметры

In [0]:
num_iterations = 20000 # @param {type:"integer"}

initial_collect_steps = 1000  # @param {type:"integer"} 
collect_steps_per_iteration = 1  # @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

## Окружающая среда

In Reinforcement Learning (RL), an environment represents the task or problem to be solved. Standard environments can be created in TF-Agents using `tf_agents.environments` suites. TF-Agents has suites for loading environments from sources such as the OpenAI Gym, Atari, and DM Control.

Загрузите среду CartPole из набора OpenAI Gym. 

In [0]:
env_name = 'CartPole-v0'
env = suite_gym.load(env_name)

Вы можете визуализировать эту среду, чтобы увидеть, как она выглядит. Свободно качающийся столб прикреплен к тележке. Цель состоит в том, чтобы переместить тележку вправо или влево, чтобы полюс был направлен вверх.

In [0]:
#@test {"skip": true}
env.reset()
PIL.Image.fromarray(env.render())

Метод `environment.step` выполняет `action` в среде и возвращает кортеж `TimeStep` содержащий следующее наблюдение за средой и вознаграждение за действие.

Метод `time_step_spec()` возвращает спецификацию для кортежа `TimeStep` . Его атрибут `observation` показывает форму наблюдений, типы данных и диапазоны допустимых значений. Атрибут `reward` показывает те же детали для вознаграждения.


In [0]:
print('Observation Spec:')
print(env.time_step_spec().observation)

In [0]:
print('Reward Spec:')
print(env.time_step_spec().reward)

Метод `action_spec()` возвращает форму, типы данных и допустимые значения допустимых действий.

In [0]:
print('Action Spec:')
print(env.action_spec())

В среде Cartpole:

- `observation` представляет собой массив из 4 поплавков:
    - положение и скорость тележки
    - угловое положение и скорость полюса
- `reward` является скалярным значением с плавающей запятой
- `action` - это скалярное целое число, имеющее только два возможных значения:
    - `0` - «двигаться влево»
    - `1` - «двигаться вправо»


In [0]:
time_step = env.reset()
print('Time step:')
print(time_step)

action = np.array(1, dtype=np.int32)

next_time_step = env.step(action)
print('Next time step:')
print(next_time_step)

Обычно создаются две среды: одна для обучения и одна для оценки. 

In [0]:
train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)

The Cartpole environment, like most environments, is written in pure Python. This is converted to TensorFlow using the `TFPyEnvironment` wrapper.

API исходной среды использует массивы Numpy. `TFPyEnvironment` преобразует их в `Tensors` чтобы сделать его совместимым с агентами и политиками Tensorflow.


In [0]:
train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

## агент

Алгоритм, используемый для решения проблемы RL, представлен `Agent` . TF-Agents предоставляет стандартные реализации различных `Agents` , в том числе:

- [DQN](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) (используется в этом уроке)
- [REINFORCE](http://www-anw.cs.umass.edu/~barto/courses/cs687/williams92simple.pdf)
- [DDPG](https://arxiv.org/pdf/1509.02971.pdf)
- [TD3](https://arxiv.org/pdf/1802.09477.pdf)
- [РРО](https://arxiv.org/abs/1707.06347)
- [SAC](https://arxiv.org/abs/1801.01290) .

Агент DQN может использоваться в любой среде, которая имеет дискретное пространство действия.

В основе агента DQN лежит `QNetwork` , модель нейронной сети, которая может научиться прогнозировать `QValues` (ожидаемые доходы) для всех действий, учитывая наблюдения из среды.

Используйте `tf_agents.networks.q_network` для создания `QNetwork` , передавая `observation_spec` , `action_spec` и кортеж, описывающий количество и размер скрытых слоев модели.


In [0]:
fc_layer_params = (100,)

q_net = q_network.QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=fc_layer_params)

Теперь используйте `tf_agents.agents.dqn.dqn_agent` для создания экземпляра `DqnAgent` . В дополнение к `time_step_spec` , `action_spec` и QNetwork для конструктора агента также требуется оптимизатор (в данном случае `AdamOptimizer` ), функция потерь и целочисленный счетчик шагов.

In [0]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

## полисы

Политика определяет, как агент действует в среде. Как правило, целью обучения с подкреплением является обучение базовой модели, пока политика не даст желаемый результат.

В этом уроке:

- Желаемый результат - держать столб в вертикальном положении на тележке.
- Политика возвращает действие (левое или правое) для каждого наблюдения `time_step` .

Агенты содержат две политики:

- `agent.policy` - основная политика, которая используется для оценки и развертывания.
- `agent.collect_policy` - вторая политика, которая используется для сбора данных.


In [0]:
eval_policy = agent.policy
collect_policy = agent.collect_policy

Политики могут быть созданы независимо от агентов. Например, используйте `tf_agents.policies.random_tf_policy` чтобы создать политику, которая будет случайным образом выбирать действие для каждого `time_step` .

In [0]:
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

Чтобы получить действие от политики, вызовите метод `policy.action(time_step)` . `time_step` содержит наблюдение из окружающей среды. Этот метод возвращает `PolicyStep` , который является именованным кортежем с тремя компонентами:

- `action` - действие, которое нужно предпринять (в данном случае `0` или `1` )
- `state` - используется для политик с состоянием (т. е. на основе RNN)
- `info` - вспомогательные данные, такие как журнал вероятностей действий

In [0]:
example_environment = tf_py_environment.TFPyEnvironment(
    suite_gym.load('CartPole-v0'))

In [0]:
time_step = example_environment.reset()

In [0]:
random_policy.action(time_step)

## Метрики и оценка

Наиболее распространенным показателем, используемым для оценки политики, является средняя доходность. Возврат представляет собой сумму вознаграждений, полученных при выполнении политики в среде для эпизода. Несколько эпизодов запускаются, создавая средний доход.

Следующая функция вычисляет среднюю доходность политики с учетом политики, среды и ряда эпизодов.


In [0]:
#@test {"skip": true}
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


# See also the metrics module for standard implementations of different metrics.
# https://github.com/tensorflow/agents/tree/master/tf_agents/metrics

Выполнение этого вычисления для `random_policy` показывает базовую производительность в среде.

In [0]:
compute_avg_return(eval_env, random_policy, num_eval_episodes)

## Буфер воспроизведения

Буфер воспроизведения отслеживает данные, собранные из среды. В этом руководстве используется `tf_agents.replay_buffers.tf_uniform_replay_buffer.TFUniformReplayBuffer` , так как он является наиболее распространенным.

Конструктор требует спецификации для данных, которые он будет собирать. Это доступно от агента, используя метод `collect_data_spec` . Размер пакета и максимальная длина буфера также необходимы.


In [0]:
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_max_length)

Для большинства агентов `collect_data_spec` - это именованный кортеж с именем `Trajectory` , содержащий спецификации для наблюдений, действий, наград и других предметов.

In [0]:
agent.collect_data_spec

In [0]:
agent.collect_data_spec._fields

## Сбор информации

Теперь выполните случайную политику в среде за несколько шагов, записав данные в буфер воспроизведения.

In [0]:
#@test {"skip": true}
def collect_step(environment, policy, buffer):
  time_step = environment.current_time_step()
  action_step = policy.action(time_step)
  next_time_step = environment.step(action_step.action)
  traj = trajectory.from_transition(time_step, action_step, next_time_step)

  # Add trajectory to the replay buffer
  buffer.add_batch(traj)

def collect_data(env, policy, buffer, steps):
  for _ in range(steps):
    collect_step(env, policy, buffer)

collect_data(train_env, random_policy, replay_buffer, steps=100)

# This loop is so common in RL, that we provide standard implementations. 
# For more details see the drivers module.
# https://www.tensorflow.org/agents/api_docs/python/tf_agents/drivers

Буфер воспроизведения теперь является коллекцией траекторий.

In [0]:
# For the curious:
# Uncomment to peel one of these off and inspect it.
# iter(replay_buffer.as_dataset()).next()

The agent needs access to the replay buffer. This is provided by creating an iterable `tf.data.Dataset` pipeline which will feed data to the agent.

Каждая строка буфера воспроизведения хранит только один шаг наблюдения. Но поскольку агенту DQN требуется как текущее, так и следующее наблюдение для вычисления потерь, конвейер набора данных будет выбирать две соседние строки для каждого элемента в пакете ( `num_steps=2` ).

Этот набор данных также оптимизирован за счет параллельных вызовов и предварительной выборки данных.

In [0]:
# Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, 
    sample_batch_size=batch_size, 
    num_steps=2).prefetch(3)


dataset

In [0]:
iterator = iter(dataset)

print(iterator)


In [0]:
# For the curious:
# Uncomment to see what the dataset iterator is feeding to the agent.
# Compare this representation of replay data 
# to the collection of individual trajectories shown earlier.

# iterator.next()

## Обучение агента

Во время цикла обучения должны произойти две вещи:

- собирать данные из окружающей среды
- использовать эти данные для обучения нейронной сети агента

Этот пример также периодически оценивает политику и печатает текущий счет.

Следующее займет ~ 5 минут для запуска.

In [0]:
#@test {"skip": true}
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

for _ in range(num_iterations):

  # Collect a few steps using collect_policy and save to the replay buffer.
  for _ in range(collect_steps_per_iteration):
    collect_step(train_env, agent.collect_policy, replay_buffer)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step = agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)

## Визуализация


### Сюжеты

Используйте `matplotlib.pyplot` чтобы составить график улучшения политики во время обучения.

Одна итерация `Cartpole-v0` состоит из 200 временных шагов. Окружающая среда дает награду `+1` за каждый шаг, на котором находится полюс, поэтому максимальная отдача за один эпизод составляет 200. Диаграммы показывают увеличение отдачи до этого максимума при каждой оценке во время тренировки. (Это может быть немного нестабильно и не увеличиваться монотонно каждый раз.)

In [0]:
#@test {"skip": true}

iterations = range(0, num_iterations + 1, eval_interval)
plt.plot(iterations, returns)
plt.ylabel('Average Return')
plt.xlabel('Iterations')
plt.ylim(top=250)

### Ролики

Графики хорошие. Но более захватывающим является то, что агент действительно выполняет задачу в среде.

Сначала создайте функцию для вставки видео в блокнот.

In [0]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

Теперь пройдитесь по нескольким эпизодам игры Cartpole с агентом. Базовая среда Python (та, что «внутри» оболочки среды TensorFlow) предоставляет метод `render()` , который выводит изображение состояния среды. Их можно собрать в видео.

In [0]:
def create_policy_eval_video(policy, filename, num_episodes=5, fps=30):
  filename = filename + ".mp4"
  with imageio.get_writer(filename, fps=fps) as video:
    for _ in range(num_episodes):
      time_step = eval_env.reset()
      video.append_data(eval_py_env.render())
      while not time_step.is_last():
        action_step = policy.action(time_step)
        time_step = eval_env.step(action_step.action)
        video.append_data(eval_py_env.render())
  return embed_mp4(filename)




create_policy_eval_video(agent.policy, "trained-agent")

Для забавы сравните обученного агента (см. Выше) с агентом, движущимся случайным образом. (Это не так хорошо.)

In [0]:
create_policy_eval_video(random_policy, "random-agent")